<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np, matplotlib.pyplot
from astropy.io import fits
import pysynphot as pyS
import pcmdpy as ppy

GPU acceleration not available, sorry
No module named 'pycuda'


In [2]:
def compute_zpts(instrument, detector, band, mjd):
    bandpass = pyS.ObsBandpass('{:s},{:s},{:s},mjd#{:d}'.format(instrument, detector, band, mjd))
    spec_bb = pyS.BlackBody(50000)
    spec_bb_norm = spec_bb.renorm(1, 'counts', bandpass)
    obs = pyS.Observation(spec_bb_norm, bandpass)
    zps = {}
    zps['vega'] = obs.effstim('vegamag')
    zps['st'] = obs.effstim('stmag')
    zps['ab'] = obs.effstim('abmag')
    return zps

In [30]:
# acs wfc1 F475W 52809
bandpass = pyS.ObsBandpass('acs,wfc1,F850lp,mjd#52809,aper#0')
spec_bb = pyS.BlackBody(50000)
spec_bb_norm = spec_bb.renorm(1.0, 'counts', bandpass)
obs = pyS.Observation(spec_bb_norm, bandpass)
obs.effstim('vegamag')

22.287894177850866

In [31]:
# acs wfc1 F475W 52809
bandpass = pyS.ObsBandpass('acs,wfc1,F475w,mjd#52809,aper#0')
spec_bb = pyS.BlackBody(50000)
spec_bb_norm = spec_bb.renorm(1.0, 'counts', bandpass)
obs = pyS.Observation(spec_bb_norm, bandpass)
obs.effstim('vegamag')

24.50234982606775

In [32]:
# acs wfc1 F475W 52809
bandpass = pyS.ObsBandpass('acs,wfc1,F850lp,mjd#52809,aper#4.0')
spec_bb = pyS.BlackBody(50000)
spec_bb_norm = spec_bb.renorm(1.0, 'counts', bandpass)
obs = pyS.Observation(spec_bb_norm, bandpass)
obs.effstim('vegamag')

24.353030715106456

In [33]:
# acs wfc1 F475W 52809
bandpass = pyS.ObsBandpass('acs,wfc1,F475w,mjd#52809,aper#4.0')
spec_bb = pyS.BlackBody(50000)
spec_bb_norm = spec_bb.renorm(1.0, 'counts', bandpass)
obs = pyS.Observation(spec_bb_norm, bandpass)
obs.effstim('vegamag')

26.174635544520704

In [57]:
def process_fits(file_name, use_proc=False):
    with fits.open(file_name) as hdu:
        header = hdu[0].header
    instrument = header['INSTRUME'].lower().strip(' ')
    detector = header['DETECTOR'].lower().strip(' ')
    if detector == 'wfc':
        detector = 'wfc1'
    band = None
    for k in ['FILTER', 'FILTER1', 'FILTER2']:
        if k in header:
            b_temp = header[k]
            if 'CLEAR' in b_temp:
                continue
            else:
                band = b_temp
                break
    if band is None:
        raise KeyError('Unable to identify filter from FITS file')
    if use_proc:
        mjd = int(header['PROCTIME'])
    else:
        mjd = int(header['EXPSTART'])
    exposure = header['EXPTIME']
    zpts = compute_zpts(instrument, detector, band, mjd)
    print('Filter: {:s}'.format(band))
    print('Observation Date: {:d} (MJD)'.format(mjd))
    print('Vega ZeroPoint: {:.4f}'.format(zpts['vega']))
    print('AB ZeroPoint: {:.4f}'.format(zpts['ab']))
    print('ST ZeroPoint: {:.4f}'.format(zpts['st']))
    print('Exposure Time: {:.1f}'.format(exposure))

In [58]:
print('B band, M51 Obs')
process_fits('../../data/h_m51_b_s05_drz_sci.fits')

B band, M51 Obs
Filter: F435W
Observation Date: 53382 (MJD)
Vega ZeroPoint: 25.7890
AB ZeroPoint: 25.6908
ST ZeroPoint: 25.1805
Exposure Time: 16320.0


In [59]:
print('g band, M31 PHAT')
process_fits('../../data/hlsp_phat_hst_acs-wfc_12105-m31-b06-f02_f475w_v1_drz.fits')

g band, M31 PHAT
Filter: F475W
Observation Date: 56122 (MJD)
Vega ZeroPoint: 26.1511
AB ZeroPoint: 26.0586
ST ZeroPoint: 25.7483
Exposure Time: 1710.0


In [60]:
print('g band, M49 Obs')
process_fits('../../data/m49/raw/m49_chip1_exp1_f475w_flc.fits')

g band, M49 Obs
Filter: F475W
Observation Date: 52809 (MJD)
Vega ZeroPoint: 26.1746
AB ZeroPoint: 26.0820
ST ZeroPoint: 25.7713
Exposure Time: 375.0


In [61]:
print('g band, M49 Obs')
process_fits('../../data/m49/raw/m49_chip2_exp2_f475w_flc.fits')

g band, M49 Obs
Filter: F475W
Observation Date: 52809 (MJD)
Vega ZeroPoint: 26.1746
AB ZeroPoint: 26.0820
ST ZeroPoint: 25.7713
Exposure Time: 375.0


In [62]:
print('V band, M51 Obs')
process_fits('../../data/h_m51_v_s05_drz_sci.fits')

V band, M51 Obs
Filter: F555W
Observation Date: 53382 (MJD)
Vega ZeroPoint: 25.7323
AB ZeroPoint: 25.7324
ST ZeroPoint: 25.6862
Exposure Time: 8160.0


In [63]:
print('I band, M51 Obs')
process_fits('../../data/h_m51_i_s05_drz_sci.fits')

I band, M51 Obs
Filter: F814W
Observation Date: 53382 (MJD)
Vega ZeroPoint: 25.5286
AB ZeroPoint: 25.9568
ST ZeroPoint: 26.7930
Exposure Time: 8160.0


In [64]:
print('I band, M31 PHAT')
process_fits('../../data/hlsp_phat_hst_acs-wfc_12105-m31-b06-f02_f814w_v1_drz.fits')

I band, M31 PHAT
Filter: F814W
Observation Date: 56121 (MJD)
Vega ZeroPoint: 25.5191
AB ZeroPoint: 25.9471
ST ZeroPoint: 26.7827
Exposure Time: 1505.0


In [65]:
print('z band, M49 Obs')
process_fits('../../data/m49/raw/m49_chip1_exp1_f850lp_flc.fits')

z band, M49 Obs
Filter: F850LP
Observation Date: 52809 (MJD)
Vega ZeroPoint: 24.3530
AB ZeroPoint: 24.8788
ST ZeroPoint: 25.9668
Exposure Time: 560.0


In [66]:
print('z band, M49 Obs')
process_fits('../../data/m49/raw/m49_chip2_exp2_f850lp_flc.fits')

z band, M49 Obs
Filter: F850LP
Observation Date: 52809 (MJD)
Vega ZeroPoint: 24.3530
AB ZeroPoint: 24.8788
ST ZeroPoint: 25.9668
Exposure Time: 560.0
